Esta notebook contiene bloques de código útiles para realizar Q-learning en el entorno "Taxi"

In [2]:
#!pip install gymnasium
#!pip install gymnasium[toy-text]
#!pip install scikit-learn
#!pip install wandb
# !pip install imageio
import numpy as np
from taxi_env_extended import TaxiEnvExtended
import matplotlib.pyplot as plt
import wandb
import sys
import os
import imageio

sys.path.append(os.path.abspath('../Models'))
from model_manager import ModelManager

Inicializar ambiente

In [ ]:
env = TaxiEnvExtended()

Obtener la cantidad de estados y acciones

In [ ]:
actions = env.action_space.n
states = env.observation_space.n

Inicialización de la tabla Q

In [ ]:
Q = np.zeros((states, actions))

Obtención de la acción a partir de la tabla Q

In [ ]:
def optimal_policy(state, Q):
    action = np.argmax(Q[state])
    return action

# es la acción que maximiza el valor de Q para un estado dado
# vamos a tener 500 observaciones, cada una con 6 acciones posibles 
# (norte, sur, este, oeste, recoger, dejar)
# para cada observación, vamos a tener el Q con las 6 acciones posibles
# entonces agarramos el máximo de las 6 acciones posibles para cada observación

Epsilon-Greedy Policy

In [ ]:
def epsilon_greedy_policy(state, Q, epsilon=0.1):
    explore = np.random.binomial(1, epsilon)
    if explore:
        action = env.action_space.sample()
        print('explore')
    else:
        action = np.argmax(Q[state])
        print('exploit')
        
    return action

# con cierta probabilidad exploramos y con cierta probabilidad explotamos
# si exploramos, tomamos una acción aleatoria
# si explotamos, tomamos la acción que maximiza el valor de Q para ese estado (lo mismo que optimal_policy)

Imprimir imagen del ambiente

In [1]:
def imgRender():
    img = env.render()
    plt.imshow(img)
    plt.axis('off')
    plt.show()

Función de entrenamiento

In [ ]:
def train(env, Q, alpha=0.1, gamma=0.9, epsilon=0.1, episodes=10):
    for i in range(episodes):
        obs,_ = env.reset()
        print(obs)
        done = False
        total_reward = 0
        step_count = 0
        while not done:
            state = obs
            action = epsilon_greedy_policy(state, Q, epsilon)
            obs, reward, done, _, _ = env.step(action)
            total_reward += reward
            step_count += 1
            Q[state, action] = Q[state, action] + alpha * (reward + gamma * np.max(Q[obs]) - Q[state, action])

        wandb.log({"total_reward": total_reward, "avg_q_value": np.mean(Q)})
        
        print('total_reward', total_reward)

# para probar el agente se usa optimal_policy, el epsilon_greedy_policy es solo para entrenar
# el objetivo es primero entrenar y después probar.

Registro con Wandb

In [ ]:
wandb.login()

Ejecución individual

In [ ]:
wandb.init(
    project="taxi",

    config={
    "alpha": 0.5,
    "gamma": 0.9,
    "epsilon": 0.1,
    "epochs": 5000,
    }
)

train(env, Q, wandb.config.alpha, wandb.config.gamma, wandb.config.epsilon, wandb.config.epochs)

wandb.finish()

Ejecución múltiple con sweep

In [ ]:
sweep_config = {
    'method': 'grid',
    'metric': {
        'name': 'reward',
        'goal': 'maximize'
    },
    'parameters': {
        'alpha': {
            'values': [0.1, 0.3, 0.5]
        },
        'gamma': {
            'values': [0.8, 0.9, 1.0]
        },
        'epsilon': {
            'values': [0.1, 0.5]
        },
        'epochs': {
            'values': [5000, 10000]
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project="taxi")

def train_sweep():
    wandb.init()
    
    config = wandb.config
    
    train(env, Q, config.alpha, config.gamma, config.epsilon, config.epochs)
    
    wandb.finish()

wandb.agent(sweep_id, function=train_sweep)


Guardar modelo entrenado

In [ ]:
manager = ModelManager()
manager.save_model('model', Q)

print("Modelo guardado en 'modelo.pkl'")

Cargar modelo entrenado

In [ ]:
manager = ModelManager()
model = manager.load_model('taxi')

Probar el modelo cargado

In [ ]:
obs,_ = env.reset()
done = False
while not done:
    state = obs
    action = optimal_policy(state, model)
    obs, reward, done, _, _ = env.step(action)
    imgRender()

Grabar video

In [ ]:
manager = ModelManager()
Q = manager.load_model('taxi')

def save_video(env, Q, video_name, num_episodes=1, fps=10):
    frames = []

    for _ in range(num_episodes):
        obs, _ = env.reset()
        done = False
        while not done:
            state = obs
            frames.append(env.render())
            action = optimal_policy(state, Q)
            obs, reward, done, _, _ = env.step(action)

    # Guardar los frames como un video
    imageio.mimsave(video_name, frames, fps=fps)


In [ ]:
env = TaxiEnvExtended()

save_video(env, Q, 'taxi_env_video.mp4', num_episodes=10, fps=5)

Cargar video a Wandb

In [ ]:
wandb.init(project='taxi')

video_name = 'taxi_env_video.mp4'

wandb.log({"Taxi execution (10 runs)": wandb.Video(video_name, fps=10, format="mp4")})

wandb.finish()
